<a href="https://colab.research.google.com/github/byu-cce270/content/blob/main/docs/unit3/01_gspread/HW_Gspread_Starter_Sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Imports
#import modules here



In [ ]:
# @title Bid Analysis




In [ ]:
# a function that selected the 3rd column of a list,
# converts it to a float and returns it for a key.
# you may have already coverted to a float - that is ok
def sort_key(row):
  return float(row[2])


In [16]:
# @title Drywall Bid Analysis
# supplied code

Drywall = Drywall.sort(key=sort_key)

# different methods
# Drywall = sorted (Drywall, key=sort_key)
# Drywall = (sorted(Drywall, key=lambda x: x[2]))



In [ ]:
# @title Concrete Bid Analysis

Concrete = Concrete.sort(key=sort_key)

# different methods
# Drywall = sorted (Drywall, key=sort_key)
# Drywall = (sorted(Drywall, key=lambda x: x[2]))





In [ ]:
# @title Client Estimating Portion

